# Introduction à PostgreSQL et PostGIS

Ce document contient une brève introduction à PostgreSQL et l'extension spatiale PostGIS. Nous commençons d'abord à regarder l'installation, puis nous faisons un tour du langage SQL et puis nous regardons PostGIS plus en détail.

La partie sur le langage SQL est en principe applicable à d'autres bases de données SQL (p.ex. MySQL, Oracle, SQLite voire Microsoft Access), même si chaque système de bases de données a quelques particularités dans son implémentation du standard SQL.

## Installation pour macOS

Il est conseillé de faire l'installation avec [Homebrew](http://brew.sh). Homebrew est un gestionnaire de paquets de logiciels, une sorte de AppStore pour les logiciels open-source et souvent lignes de commande.

Généralement, Homebrew aura besoin de XCode pour installer les différents paquets. Xcode peut être installé à l'aide de l'AppStore d'Apple.

Une fois Homebrew installé, il suffit d'ouvrir le Terminal et de lancer l'installation comme suit:

    brew update
    brew install postgresql postgis
    
ce qui va prendre un peu de temps...

Homebrew peut aussi être utilisé pour faire des mises à jour (`brew upgrade postgresql`). Attention avec PostgreSQL: la mise à jour d'une version majeur (p.ex. 9.5 à 9.6) nécessite l'exportation complète de la base de données avant la mise à jour (p.ex. avec `pg_dumpall`).

## Installation d'une GUI

PostgreSQL vient par défaut avec une série de logiciels ligne de commande, dont `psql` pour exécuter les instructions SQL, `pg_dump` et `pg_dumpall` pour sauvegarder une base de données, `createdb` pour créer une nouvelle base de données, ou `dropdb` pour en détruire une.

Ces logiciels ligne de commande sont en principe suffisants pour exécuter l'ensemble des tâches, mais il est souvent plus comfortable d'utiliser un logiciel avec interface graphique (GUI) pour exécuter une partie des tâches.

Il y a plusieurs logiciels de ce genre pour PostgreSQL, dont:

- [pgAdmin4](http://pgadmin.org) qui est le GUI le plus populaire, mais pas forcément le plus convivial

et pour macOS:

- [Postico](https://eggerapps.at/postico/) qui est très convivial mais payant si on veut utiliser plus de 5 connexions de bases de données (nous en aurons besoin en principe d'une seule)
- [PSequel](http://www.psequel.com/)

Pour macOS, je recommande d'utiliser pgAdmin4 et Postico, car pgAdmin4 est plus complet, mais Postico plus convivial.


## Créer une nouvelle base de données

Il y a plusieurs façons de créer une nouvelle base de données PostgreSQL. Une possibilité est d'utiliser l'outil de ligne de commande `createdb`, comme p.ex. ici une base de données avec nom `votations_suisses`:

    createdb votations_suisses

L'aide peut être affichée avec

    createdb --help
    
Il se peut, suivant le mode d'installation, qu'il faut régler les paramètres d'accès à la base de données, et/ou indiquer un nom d'utilisateur et mot de passe. Il peut p.ex. être nécessaire de créer un utilisateur qui a le même nom que l'utilisateur système (créer avec `createrole monutilisateur`) et même ajuster les paramètres dans le fichier `pg_hba.conf` (localisé suivant comment dans le dossier `/usr/local/var/postgres`, dossier à trouver avec `ps aux | grep postgres`; et surtout demander de l'aide dans ce cas, c'est déjà du Postgres assez avancé...).

## Créer une table et importer quelques données

Nous allons dans un premier temps créer une table manuellement, ainsi que d'importer quelques données. Nous allons travailler sur les données de la votation du 12 février 2017 sur la réforme de l'imposition fiscale. Le tableau de données est disponible sur le site Web de l'OFS ([ici](https://www.bfs.admin.ch/bfs/fr/home/statistiques/politique/votations/annee-2017/2017-02-12/reforme-imposition-entreprises.assetdetail.1866514.html)), et une variante simplifiée et converti en fichier TSV est [ici](https://raw.githubusercontent.com/christiankaiser/geovis2/master/data/votation_120217_reforme_fiscale.tsv).

Nous allons convertir ce fichier en commandes SQL pour l'importer dans la base de données. Nous pouvons créer la table de la manière suivante:

    CREATE TABLE votation_120217_reforme_fiscale (
        id_canton varchar(2) PRIMARY KEY,
        nom_canton varchar(60) NOT NULL,
        electeurs_inscrits integer,
        bulletins_rentres integer,
        participation decimal(5,2),
        blancs integer,
        nuls integer,
        bulletins_valables integer,
        oui integer,
        non integer,
        oui_pourcentage decimal(5,2)
    );
    
L'insertion du premier canton peut se faire comme suit:

    INSERT INTO votation_120217_reforme_fiscale 
        (id_canton, nom_canton, electeurs_inscrits, 
         bulletins_rentres, participation, blancs, nuls, 
         bulletins_valables, oui, non, oui_pourcentage) 
    VALUES (
        'ZH', 'Zürich', 920475, 446479, 48.5, 8498, 
        120, 437861, 164319, 273542, 37.5
    );

La requête SQL peut être écrite sur une ou plusieurs lignes. L'indication des champs de la table est facultative si la liste des valeurs est complète et dans le bon ordre.

L'insertion en bloc de tous les canton est possible sans trop de problèmes. Il suffit d'éditer le fichier des données en conséquence, en utilisant du rechercher/remplacer intelligent (utiliser p.ex. Grep dans TextWrangler) ou l'édition simultanée de toutes les lignes en synchrone (utiliser les curseurs multiples p.ex. de SublimeText ou Atom).

Essayez p.ex. de faire le rechercher/remplacer suivant dans TextWrangler après avoir enlevé la première ligne du fichier TSV (activer l'option Grep):

- Texte à chercher: `(.*?)\t(.*?)\t(.*?)\t(.*?)\t(.*?)\t(.*?)\t(.*?)\t(.*?)\t(.*?)\t(.*?)\t(.*?)\r`
- Remplacer par: `INSERT INTO votation_120217_reforme_fiscale VALUES ('\1', '\2', \3, \4, \5, \6, \7, \8, \9, \10, \11);\r`

Consultez l'aide de TextWrangler pour plus d'informations sur cette magie (menu Help > User Manual; voir les 2 chapitres sur la recherche de texte.

Il suffit de copier/coller les requêtes SQL...

## Sélection de données

Nous pouvons désormais faire des requêtes de sélection de données, p.ex. la liste des cantons qui a dit oui à la réforme fiscale, trié par le nombre de bulletins «oui»:

    SELECT * FROM votation_120217_reforme_fiscale 
    WHERE oui_pourcentage > 50
    ORDER BY oui;
    
et le nombre total de bulletins oui et non, ainsi que la proportion de oui des cantons romands:

    SELECT SUM(oui) AS oui, SUM(non) AS non, SUM(oui) * 100.0 / SUM(bulletins_valables) AS prop_oui
    FROM votation_120217_reforme_fiscale
    WHERE id_canton IN ('GE', 'VD', 'NE', 'VS', 'FR', 'JU');





## Modifier la table et les données

Il est bien évidemment possible de modifier la structure des tables et d'éditer les données avec SQL. Nous pouvons par exemple ajouter un champ pour la région linguistique:

    ALTER TABLE votation_120217_reforme_fiscale 
    ADD COLUMN region_linguistique varchar(2);
    
et puis modifier les données par groupes:

    UPDATE votation_120217_reforme_fiscale 
    SET region_linguistique = 'de';
    
    UPDATE votation_120217_reforme_fiscale 
    SET region_linguistique = 'fr'
    WHERE id_canton IN ('GE', 'VD', 'NE', 'VS', 'FR', 'JU');
    
    UPDATE votation_120217_reforme_fiscale 
    SET region_linguistique = 'it'
    WHERE id_canton = 'TI';

Vous pouvez exécuter plusieurs requêtes SQL à la fois. Si une erreur survient dans une des requêtes, aucun changement même des requêtes précédentes n'est retenu dans la base de données. PostgreSQL traite toutes les requêtes exécutées en même temps comme un seul bloc qui est appelé __«une transaction»__. Ce comportement diffère d'autres bases de données comme p.ex. MySQL. En raison de cette propriété, PostgreSQL est une __base de données transactionnelle__.

## Agrégation dynamique

Le SQL contient une instruction `GROUP BY` qui permet d'agréger des données, par exemple par région linguistique:

    SELECT region_linguistique, SUM(oui) AS oui, SUM(oui) * 100.0 / SUM(bulletins_valables) AS prop_oui
    FROM votation_120217_reforme_fiscale
    GROUP BY region_linguistique;

Pour les champs qui ne se trouvent pas dans l'instruction `GROUP BY`, il faut alors utiliser un opérateur d'agrégation, comme p.ex. `SUM`, `MEAN`, `MAX` ou `MIN`.

Il est possible de faire un `GROUP BY` sur plusieurs colonnes.


## Importation d'un fichier Shape

L'extension PostGIS insère un champ de type `GEOMETRY` dans PostgreSQL ainsi que toute une série de fonctions qui permettent de travailler avec ce champ. La géometrie stockée dans le champ peut être un point, ligne ou polygone, et elle a un système de référence spatial associé.

Dans un premier temps, il faut activer l'extension PostGIS au niveau de la base de données:

    CREATE EXTENSION postgis;

Par la suite, il est possible d'importer par exemple des fichiers Shape dans une nouvelle table PostGIS. Plusieurs façons de faire existent (p.ex. à l'aide de plugins QGIS), nous allons utiliser ici l'utilitaire en ligne `shp2pgsql`. L'aide de l'utilitaire peut être affichée [vec

    shp2pgsql -h

`shp2pgsql` lit un fichier Shape et produit les instructions SQL nécessaires pour créer la table correspondante et pour insérer les données contenues dans le fichier Shape.

Comme exemple, nous utilisons le fichier `cantons.shp`, téléchargeable depuis [ici en format ZIP](). Après décompression, nous pouvons l'importer en utilisant l'instruction suivante:

    shp2pgsql -s EPSG:21781 -I -W "ISO-8859-1" cantons.shp cantons > cantons.sql

L'option `-s EPSG:21781` définit le système de référence spatial du fichier Shape d'entrée. L'option `-I` crée un index pour le champ de la géometrie, ce qui permet d'accélérer les requêtes spatiales. `-W "ISO-8859-1"` définit l'encodage utilisé dans le fichier Shape pour les attribut; c'est important uniquement pour les accents.

Par la suite, nous pouvons exécuter les instructions du fichier `cantons.sql` pour obtenir la table correspondante dans PostGIS. Inspéctez la nouvelle table, et regardez notamment le format du champ de géométrie. PostGIS utilise un format binaire pour stocker les géométries!

La requête suivante calcule la superficie de chaque canton en hectares (grâce à la division par 10'000):

    SELECT canton, ST_AREA(geom) / 10000 AS superficie FROM cantons;

Comment on peut trouver le canton le plus grand?

    SELECT canton, ST_Area(geom) / 10000 AS superficie FROM cantons
    ORDER BY superficie DESC
    LIMIT 1;
    
 

## Jointure de tables

Nos deux tables `cantons` et `votation_120217_reforme_fiscale` peuvent être liées à l'aide de l'`id_canton`. Il s'agit d'une relation 1:1; la base de données n'est donc pas normalisée. Nous gardons tout de même cette structure des tables pour des raisons pratiques.

PostgreSQL supporte la définition de relations explicites entre tables, mais nous pouvons procéder aux jointures aussi sans cette définition explicite.

Il y a plusieurs façons d'écrire une jointure entre nos deux tables, en voici deux qui sont équivalentes (il s'agit d'une jointure interne):

    SELECT *
    FROM cantons JOIN votation_120217_reforme_fiscale 
    ON cantons.id_canton = votation_120217_reforme_fiscale.id_canton;

ou encore:

    SELECT * FROM cantons, votation_120217_reforme_fiscale 
    WHERE cantons.id_canton = votation_120217_reforme_fiscale.id_canton;

Pour sélectionner uniquement une partie des champs, on peut écrire:

    SELECT cantons.id_canton, nom_canton, canton_no, geom
    FROM cantons JOIN votation_120217_reforme_fiscale 
    ON cantons.id_canton = votation_120217_reforme_fiscale.id_canton;

Il faut noter que pour des champs comme `id_canton` qui se trouvent dans les deux tables, il faut indiquer également le nom de la table. Ceci peut mener à des requêtes relativement longues, surtout avec des noms de tables compliqués. Un solution est de définir des __alias__ pour les noms de tables. Voici la même requêtes avec alias:

    SELECT C.id_canton, nom_canton, canton_no, geom
    FROM cantons C JOIN votation_120217_reforme_fiscale V
    ON C.id_canton = V.id_canton;

ce qui est un peu plus lisible. Un alias peut être n'importe quel nom qui n'entre pas en conflit avec les noms déjà existants.

Les __instructions SQL ne sont pas sensibles à la casse__, mais pour les noms des tables et champs, il est important de respecter les majuscules et minuscules, car sur certains systèmes, une différence est faite. Généralement, il est une bonne pratique d'écrire les instructions SQL en majuscules, et les noms de champs et tables en minuscules.


## Quelques requêtes spatiales

La force de PostGIS est la possibilité d'exécuter des requêtes spatiales sur les géométries. Voici quelques exemples.

- Pour __projeter__ les cantons suisses en un autre SRS, il suffit d'écrire

        SELECT id_canton, canton, ST_Transform(geom, 4326) AS geom FROM cantons;

- Pour __convertir__ les géométries en format __Well-Known Text (WKT)__:

        SELECT id_canton, canton, ST_AsText(geom) AS geom FROM cantons;
  
  respectivement après projection:
  
        SELECT id_canton, canton, ST_AsText(ST_Transform(geom, 4326)) AS geom FROM cantons;

- Ou une conversion en format __GeoJSON__:

        SELECT id_canton, canton, ST_AsGeoJson(geom) AS geom FROM cantons;

- Calculer une __zone tampon__ de 1km autour le canton de Berne:

        SELECT ST_Buffer(geom, 1000) FROM cantons WHERE id_canton = 'BE';

- Sélectionner les cantons qui sont à moins de 20 km du canton de Berne:

        SELECT C.id_canton, C.canton
        FROM cantons C, (SELECT * FROM cantons WHERE id_canton = 'BE') B
        WHERE ST_Distance(C.geom, B.geom) <= 20000
  
  Il y a au total 16 cantons (avec le canton de Berne) qui sont à moins de 20 km du canton de Berne. C'est surprenant d'y voir des cantons comme les Grisons ou le Tessin, mais une vérification dans QGIS permet de le confirmer... Affichons également la distance et faisons un tri selon la distance:
  
        SELECT C.id_canton, C.canton, ST_Distance(C.geom, B.geom) AS d
        FROM cantons C, (SELECT * FROM cantons WHERE id_canton = 'BE') B
        WHERE ST_Distance(C.geom, B.geom) <= 20000
        ORDER BY d, C.canton;


Ce dernier exemple utilise le concept de __sous-requête__. En effet, dans la clause `FROM`, nous trouvons la table des cantons, puis un résultat d'une autre requête. Cette sous-requête est écrite entre parenthèses est possède obligatoirement un __alias__. Par ailleurs, on peut la traiter comme n'importe quelle autre table!



## Les vues

Il est possible «d'enregistrer» une requête SQL de type `SELECT` dans ce qu'on appelle une `VIEW`. Si nous avons p.ex. la requête 

    SELECT id_canton, canton, ST_Transform(geom, 4326) AS geom, ST_AREA(geom) AS superficie FROM cantons;

nous pouvons créer une vue correspondante avec:

    CREATE VIEW cantons_wgs84 AS
    SELECT id_canton, canton, ST_Transform(geom, 4326) AS geom, ST_AREA(geom) AS superficie FROM cantons;

Une vue peut être traitée comme une table normale, sauf que la requête est exécutée à chaque fois que nous en avons besoin. Le contenu est donc dynamique, c'est-à-dire si on met à jour les données dans la table d'origine, le contenu de la vue est mis à jour aussi.

Nous pouvons par ailleurs aussi créer une nouvelle table à partir d'une requête SQL, sans contenu dynamique:

    CREATE TABLE cantons_wgs84_permanent AS
    SELECT id_canton, canton, ST_Transform(geom, 4326) AS geom, ST_AREA(geom) AS superficie FROM cantons;

et puis si vous voulez la supprimer à nouveau:

    DROP TABLE cantons_wgs84_permanent;
    
__Attention:__ Cette action ne vous demandera aucune confirmation, ne montrera aucun message de mise en garde, rien. Elle va juste supprimer la table très rapidement et sans poser aucune question. Pratique pour perdre des données...  
Conséquence: __faites souvent des copies de sauvegardes avec `pg_dump`__.

Les tables avec champ de géométrie, et aussi les vues avec champ de géométrie, peuvent être ouvertes dans QGIS de la même manière que n'importe quelle autre couche vectorielle.

Les tables peuvent aussi être éditées avec QGIS avec les outils d'édition standard.
